In [ ]:
!pip install kagglehub
!pip install opencv-contrib-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 190.9 MB/s eta 0:00:00a 0:00:01
ERROR: Could not find a version that satisfies the requirement ultralythics (from versions: none)
ERROR: No matching distribution found for ultralythics


In [4]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 199.4 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
import shutil
import os
import kagglehub

# Download the dataset
dataset_path = kagglehub.dataset_download('sebastianpalaciob/plantvillage-for-object-detection-yolo')

# Destination path you can see in the file explorer
visible_path = '/teamspace/studios/this_studio/plantvillage'

# Create the destination folder if it doesn't exist
os.makedirs(visible_path, exist_ok=True)

# Copy the entire dataset folder to visible directory
shutil.copytree(dataset_path, visible_path, dirs_exist_ok=True)

print("Copied dataset to visible path:", visible_path)


100%|██████████| 829M/829M [00:08<00:00, 102MB/s]  

Extracting files...


Copied dataset to visible path: /teamspace/studios/this_studio/plantvillage


In [5]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from PIL import Image
import random
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics 8.3.111 🚀 Python-3.10.10 torch-2.2.1+cu121 CPU (Intel Xeon Platinum 8175M 2.50GHz)
Setup complete ✅ (4 CPUs, 15.2 GB RAM, 30.8/387.5 GB disk)


In [6]:
img_path='/teamspace/studios/this_studio/plantvillage/PlantVillage_for_object_detection/Dataset/images'
label_path='/teamspace/studios/this_studio/plantvillage/PlantVillage_for_object_detection/Dataset/labels'

In [8]:
ipaths=[]
types=[]
for dirname, _, filenames in os.walk(img_path):
    for filename in filenames:
        ipaths+=[os.path.join(dirname, filename)]
        types+=[filename.split('.')[-1]]
tpaths=[]
for dirname, _, filenames in os.walk(label_path):
    for filename in filenames:
        tpaths+=[os.path.join(dirname, filename)]    
ipaths=sorted(ipaths)
tpaths=sorted(tpaths)
paths=[]
for ip,tp in zip(ipaths,tpaths):
    paths+=[(ip,tp)]
random.shuffle(paths)
ipaths=[]
tpaths=[]
for p in paths:
    ipaths+=[p[0]]
    tpaths+=[p[1]]

In [9]:
print(len(ipaths))

54293


In [10]:
print(set(types))
for typei in list(set(types)):
    print(typei,types.count(typei))

{'jpg', 'JPG'}
jpg 54292
JPG 1


In [11]:
boxdata=[]
boxfile=[]
for i in range(len(tpaths)):
    tfile=tpaths[i]
    ifile=ipaths[i]
    boxdata+=[np.loadtxt(tfile)]     
    boxfile+=[ifile.split('/')[-1]]
print(boxdata[0])

[         35     0.46484     0.48242     0.60156     0.91797]


In [12]:
BOX=pd.DataFrame()

for i in range(len(boxdata)):
    if type(boxdata[i][0])==np.float64:
        add=pd.DataFrame([boxdata[i]])
        add[5]=boxfile[i]
        BOX=pd.concat([BOX,add])
    else:
        add=pd.DataFrame(boxdata[i])
        add[5]=boxfile[i]
        BOX=pd.concat([BOX,add])       

BOX2=BOX.reset_index(drop=True)
BOX2.iloc[:,0]=BOX2.iloc[:,0].astype(int)
display(BOX2)

,0,1,2,3,4,5
0,35.0,0.464844,0.482422,0.601562,0.917969,TMYC_image (4576).jpg
1,35.0,0.505859,0.503906,0.878906,0.937500,TMYC_image (1978).jpg
2,5.0,0.513672,0.500000,0.933594,0.992188,CHPM_image (49).jpg
3,2.0,0.476562,0.509766,0.914062,0.949219,APCR_image (149).jpg
4,20.0,0.488281,0.492188,0.976562,0.984375,PTEB_image (117).jpg
...,...,...,...,...,...,...
54343,21.0,0.482422,0.494141,0.933594,0.980469,PTLB_image (230).jpg
54344,24.0,0.453125,0.521484,0.906250,0.910156,SOHE_image (3588).jpg
54345,28.0,0.490234,0.500000,0.964844,0.992188,TMBS_image (192).jpg
54346,35.0,0.460938,0.503906,0.820312,0.773438,TMYC_image (4515).jpg


In [13]:
display(BOX2.iloc[:,0].value_counts())

0
15.0    5507
35.0    5361
24.0    5089
16.0    2297
28.0    2127
30.0    1911
25.0    1835
32.0    1778
33.0    1677
3.0     1645
37.0    1591
4.0     1502
19.0    1477
34.0    1405
12.0    1383
8.0     1192
11.0    1180
10.0    1157
26.0    1134
13.0    1076
5.0     1052
29.0    1004
20.0    1000
21.0    1000
18.0     997
9.0      989
31.0     952
6.0      853
0.0      630
1.0      621
7.0      516
27.0     456
14.0     423
36.0     373
23.0     371
17.0     360
2.0      275
22.0     152
Name: count, dtype: int64

In [14]:
BOX2[5].value_counts()

5
STLS_image (343).jpg     3
TMEB_image (967).jpg     3
CNLB_image (339).jpg     2
TMSL_image (1141).jpg    2
STLS_image (279).jpg     2
                        ..
TMLM_image (649).jpg     1
TMLM_image (439).jpg     1
TMBS_image (1796).jpg    1
APHE_image (642).jpg     1
ORHL_image (1592).jpg    1
Name: count, Length: 54293, dtype: int64

In [15]:
def draw_box(n0):
    
    ipath=ipaths[n0]
    image=cv2.imread(ipath)
    H,W=image.shape[0],image.shape[1]
    file=ipath.split('/')[-1] 
    
    if BOX2[BOX2[5]==file] is not None:
        box=BOX2[BOX2[5]==file]
        box=box.reset_index(drop=True)
        #display(box)
        
        for i in range(len(box)):
            label=int(box.loc[i,0])
            x=box.loc[i,1]
            y=box.loc[i,2]
            w=box.loc[i,3] 
            h=box.loc[i,4]
            x1=((x-w/2)*W).astype(int)
            y1=((y-h/2)*H).astype(int)
            x2=((x+w/2)*W).astype(int)
            y2=((y+h/2)*H).astype(int)
            
            cv2.rectangle(image,(x1,y1),(x2,y2),(0,255,0),2) #green

            cv2.putText(image, f'{label}', (50,50), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)     
    #plt.imshow(image)
    #plt.show()
    
    return image

In [16]:
images1=[]
for i in range(len(ipaths)):
    images1+=[draw_box(i)]

In [ ]:
from matplotlib import animation, rc
rc('animation', html='jshtml')

In [ ]:
def create_animation(ims):    
    fig=plt.figure(figsize=(10,6))
    #plt.axis('off')
    im=plt.imshow(cv2.cvtColor(ims[0],cv2.COLOR_BGR2RGB))
    plt.close()    
    def animate_func(i):
        im.set_array(cv2.cvtColor(ims[i],cv2.COLOR_BGR2RGB))
        return [im]
    return animation.FuncAnimation(fig, animate_func, frames=len(ims), interval=1000//2)

In [ ]:
create_animation(images1)

In [17]:
os.makedirs('datasets', exist_ok=True)
os.makedirs('datasets/train', exist_ok=True)
os.makedirs('datasets/valid', exist_ok=True)
os.makedirs('datasets/test', exist_ok=True)

In [18]:
from shutil import copyfile

total = len(ipaths)

for i, (ipath, tpath) in enumerate(zip(ipaths, tpaths)):
    ifile = ipath.split('/')[-1]
    tfile = tpath.split('/')[-1]

    ratio = i / total

    if ratio < 0.7:
        folder = 'train'
    elif ratio < 0.9:
        folder = 'valid'
    else:
        folder = 'test'

    copyfile(ipath, f'datasets/{folder}/{ifile}')
    copyfile(tpath, f'datasets/{folder}/{tfile}')


In [19]:
import yaml

class_names = [
    'Apple___Apple_scab',
    'Apple___Black_rot',
    'Apple___Cedar_apple_rust',
    'Apple___healthy',
    'Blueberry___healthy',
    'Cherry___Powdery_mildew',
    'Cherry___healthy',
    'Corn___Cercospora_leaf_spot Gray_leaf_spot',
    'Corn___Common_rust',
    'Corn___Northern_Leaf_Blight',
    'Corn___healthy',
    'Grape___Black_rot',
    'Grape___Esca_(Black_Measles)',
    'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
    'Grape___healthy',
    'Orange___Haunglongbing_(Citrus_greening)',
    'Peach___Bacterial_spot',
    'Peach___healthy',
    'Pepper,_bell___Bacterial_spot',
    'Pepper,_bell___healthy',
    'Potato___Early_blight',
    'Potato___Late_blight',
    'Potato___healthy',
    'Raspberry___healthy',
    'Soybean___healthy',
    'Squash___Powdery_mildew',
    'Strawberry___Leaf_scorch',
    'Strawberry___healthy',
    'Tomato___Bacterial_spot',
    'Tomato___Early_blight',
    'Tomato___Late_blight',
    'Tomato___Leaf_Mold',
    'Tomato___Septoria_leaf_spot',
    'Tomato___Spider_mites Two-spotted_spider_mite',
    'Tomato___Target_Spot',
    'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
    'Tomato___Tomato_mosaic_virus',
    'Tomato___healthy'
]

face_yaml = dict(
    train='train',
    val='valid',
    test='test',
    nc=len(class_names),
    names=class_names
)

with open('face.yaml', 'w') as outfile:
    yaml.dump(face_yaml, outfile, default_flow_style=True)
    
%cat face.yaml

{names: [Apple___Apple_scab, Apple___Black_rot, Apple___Cedar_apple_rust, Apple___healthy,
    Blueberry___healthy, Cherry___Powdery_mildew, Cherry___healthy, Corn___Cercospora_leaf_spot
      Gray_leaf_spot, Corn___Common_rust, Corn___Northern_Leaf_Blight, Corn___healthy,
    Grape___Black_rot, Grape___Esca_(Black_Measles), Grape___Leaf_blight_(Isariopsis_Leaf_Spot),
    Grape___healthy, Orange___Haunglongbing_(Citrus_greening), Peach___Bacterial_spot,
    Peach___healthy, 'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy', Potato___Early_blight,
    Potato___Late_blight, Potato___healthy, Raspberry___healthy, Soybean___healthy,
    Squash___Powdery_mildew, Strawberry___Leaf_scorch, Strawberry___healthy, Tomato___Bacterial_spot,
    Tomato___Early_blight, Tomato___Late_blight, Tomato___Leaf_Mold, Tomato___Septoria_leaf_spot,
    Tomato___Spider_mites Two-spotted_spider_mite, Tomato___Target_Spot, Tomato___Tomato_Yellow_Leaf_Curl_Virus,
    Tomato___Tomato_mosaic_virus, Tomato__

In [20]:
model = YOLO("yolo11n.pt")

100%|██████████| 5.35M/5.35M [00:00<00:00, 158MB/s]


In [ ]:
import subprocess

subprocess.run([
    "yolo", "detect", "train",
    "model=yolo11n.pt",
    "data=face.yaml",
    "epochs=200",
    "imgsz=640",
    "hsv_h=0.015", "hsv_s=0.7", "hsv_v=0.4",
    "degrees=5", "translate=0.1", "scale=0.5", "shear=2.0",
    "perspective=0.002", "flipud=0.3", "fliplr=0.5",
    "mosaic=1.0", "mixup=0.2", "copy_paste=0.1",
    "close_mosaic=15", "cache=True"
])
